# Np's and Mesogens, Temperature Ramp 6.8 -> 7.0

In [1]:
#-----Import essential packages.

from __future__ import division
import hoomd
import hoomd.md

In [2]:
#-----Define important variables.

p_max = 1.8;
t_0   = 6.8; 
t_1   = 7.0;
steps_ramp = 5e4;

In [3]:
#-----Declare the file from wich we'll extract the configuration of the system.

ramp_file = "T_CM&NP_" + str(t_0) + "_P_" + str(p_max) + "_ramp.gsd"

In [4]:
#-----Declare a simulation context.

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Initialize from a .gsd file.

system = hoomd.init.read_gsd(ramp_file, frame = -1);

#-----Define each mesogen in the local frame of reference of each rigid body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types = ['A']*8,
                positions = [(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Create the mesogens.

rigid.create_bodies();

#-----Define the potential energy.

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode = 'shift')

#-----Declare interactions.

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define some groups and make their union.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#----Integrate using an NPT integrator.

temp = hoomd.variant.linear_interp(points = [(0,t_0), (steps_ramp, t_1)]);

npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = temp, tau = 2.0, tauP = 3.0, P = p_max);
npt.randomize_velocities(seed=42)

#-----Save .log y .gsd data.

log_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM&NP_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = groupNP_mes,
               overwrite = True); 

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1004 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [5]:
#-----Run the simulation.

hoomd.run(steps_ramp)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9004
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 951201 / 1001000 | TPS 19.9583 | ETA 00:41:35
Time 00:00:20 | Step 951383 / 1001000 | TPS 18.1829 | ETA 00:45:28
Time 00:00:30 | Step 951600 / 1001000 | TPS 21.6762 | ETA 00:37:58
Time 00:00:40 | Step 951778 / 1001000 | TPS 17.7742 | ETA 00:46:09
Time 00:00:50 | Step 951957 / 1001000 | TPS 17.8268 | ETA 00:45:51
Time 00:01:00 | Step 952130 / 1001000 | TPS 17.2322 | ETA 00:47:15
Time 00:01:10 | Step 952307 / 1001000 | TPS 17.6528 | ETA 00:45:58
Time 00:01:20 | Step 952470 / 1001000 | TPS 16.2754 | ETA 00:49:41
Time 00:01:30 | Step 952665 / 1001000 | TPS 19.404 | ETA 00:41:30
Time 00:01:40 | Step 952866 / 1001000 | TPS 19.9227 | ETA 00:40:16
Time 00:01:50 | Step 953061 / 1001000 | TPS 19.4752 | ETA 00:41:01
Time 00:02:00 | Step 

In [6]:
#-----Get density.

system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x10b66a8d0>),
             ('particles', <hoomd.data.particle_data at 0x10b6d9e10>),
             ('number_density', 0.535380267425981),
             ('bonds', <hoomd.data.bond_data at 0x10b6d9e80>),
             ('angles', <hoomd.data.angle_data at 0x10b6d9ef0>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x10b6d9f60>),
             ('impropers', <hoomd.data.dihedral_data at 0x10b6d9fd0>),
             ('constraints', <hoomd.data.constraint_data at 0x10b792080>),
             ('pairs', <hoomd.data.bond_data at 0x10b7920f0>),
             ('timestep', 1001000)])

In [7]:
#-----Get volume.

system.box.get_volume()

16817.9526736944